### https://www.opinet.co.kr/user/main/mainView.do 사이트에서 셀레니움 라이브러리를 활용하여 대전의 주유소 보통 휘발유 가격을 기록한 엑셀 파일을 저장하는 코드를 작성하시오.

In [76]:
from urllib.request import urlopen
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [77]:
driver = webdriver.Chrome('chromedriver')
time.sleep(3)
driver.get('https://www.opinet.co.kr')
time.sleep(3)

#자주 찾는 메뉴 > 지역별 주유소 통계
driver.find_element_by_css_selector("#quick_ul > li:nth-child(2)").click()
time.sleep(3)

# 지역 시/도 select tag에 '대전' 입력
driver.find_element_by_css_selector("#SIDO_NM0").send_keys('대전')
time.sleep(3)

# 지역 시/군/구 option tag의 value 추출
_sigungu = driver.find_elements_by_css_selector("#SIGUNGU_NM0 > option")
gu_list = [gu.get_attribute('value') for gu in _sigungu if gu.get_attribute('value') != '']
print(gu_list)

['대덕구', '동구', '서구', '유성구', '중구']


In [78]:
values = []

for gu in gu_list:
    # 지역 시/군/구 option tag에 각 행정구역 입력
    driver.find_element_by_css_selector("#SIGUNGU_NM0").send_keys(gu)
    time.sleep(3)
    
    # 조회 버튼 클릭
    driver.find_element_by_id("searRgSelect").click()
    time.sleep(3)
    
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # 조회 결과에서 주유소 이름과 휘발유 가격 파싱
    names = [name.text.strip() for name in soup.select('tbody#body1 a')]
    prices = [price.text.strip() for price in soup.select('#body1 td.rlist + .price')]
    
    values.extend(list(zip(names, prices)))
print(values)

[('㈜에스에이치에너지...', '1583'), ('일등주유소', '1589'), ('금강주유소', '1595'), ('대덕주유소', '1595'), ('소태산주유소', '1595'), ('신탄진알뜰주유소', '1597'), ('신탄진(상)주유소...', '1602'), ('CJ대한통운㈜ 대...', '1605'), ('쌍용주유소', '1605'), ('성찬셀프주유소', '1606'), ('새로운주유소', '1608'), ('한라주유소', '1609'), ('순우리주유소', '1613'), ('중도석유(주)오정...', '1615'), ('동대전셀프주유소', '1615'), ('신탄진IC주유소', '1615'), ('스마일주유소', '1616'), ('용은주유소', '1616'), ('씨앤에스유통(주)...', '1619'), ('으뜸회덕주유소', '1619'), ('씨앤에스유통(주)...', '1625'), ('신탄진농협주유소', '1625'), ('원촌주유소', '1625'), ('은빛주유소', '1625'), ('㈜연정 에스에스주...', '1625'), ('와동주유소', '1625'), ('일등주유소', '1627'), ('갑천주유소', '1628'), ('덕암셀프주유소', '1629'), ('㈜연정 농수산주유...', '1629'), ('대전북부새마을금고...', '1629'), ('중리셀프주유소', '1629'), ('SK주홍셀프주유소', '1629'), ('제일셀프주유소', '1629'), ('㈜학하 목상셀프주...', '1635'), ('세종Self주유소', '1635'), ('삼영주유소', '1638'), ('OK주유소', '1638'), ('상서주유소', '1639'), ('지에스칼텍스㈜ 톨...', '1644'), ('현대오일뱅크㈜직영...', '1645'), ('(주)메이저플러스...', '1649'), ('현대오일뱅크㈜직영...', '1649'), ('동양주유소', '1655'), ('재하주유소', '1655'), ('당산주유소', '1

In [79]:
columns = ['이름', '가격']
df = pd.DataFrame(values, index=[i+1 for i in range(len(values))], columns = columns)
df.to_excel('주유소.xlsx')